In [96]:
import pandas as pd

train = pd.read_csv("p2_data/data_train.csv", header=None)
target = train.shape[1]-1
y = train[target].to_numpy()
X = train.drop(columns=[target]).to_numpy()

test = pd.read_csv("p2_data/data_test.csv", header=None)
y_test = test[target]
X_test = test.drop(columns=[target])


In [97]:
from sklearn import tree
import random
from sklearn.utils import resample
import numpy as np

class RandomForest:
    def __init__(self, n, depth):
        self.n = n
        self.depth = depth
        self.trees = []
        self.features = []
        self.samplesX = []
        self.samplesy = []
        self.n_classes = 2
        self.build()
        
    def build(self):
        for i in range(self.n):
            clf = tree.DecisionTreeClassifier(max_depth=self.depth)
            self.trees.append(clf)
            
    def fit(self, X, y, n):
        self.n_classes = len(np.unique(y))
        for i in range(self.n):
            self.bootstrap(X, y)
            self.random_features(n, X.shape[1]-1)
            self.trees[i] = self.trees[i].fit(self.samplesX[i][:, self.features[i]], self.samplesy[i])
            
    def predict(self, X):
        predictions = np.zeros(shape=(1, self.n_classes))
        for i in range(self.n):
            predictions = np.add(predictions,self.trees[i].predict_proba(X[self.features[i]].reshape(1, -1)))
            
        return np.argmax(predictions)

    def bootstrap(self, X, y):
        r = random.randint(0, 1000)
        bootX = resample(X, replace=True, n_samples=len(X), random_state=r)
        booty = resample(y, replace=True, n_samples=len(X), random_state=r)
        self.samplesX.append(bootX)
        self.samplesy.append(booty)
        
    def random_features(self, n, n_features):
        features = []
        for i in range(n):
            features.append(random.randint(0, n_features))
        self.features.append(features)


In [98]:
clf = RandomForest(15, 3)
clf.fit(X, y, 3)

In [101]:
print(y[0])
print(clf.predict(X[0, :]))

8
8
